<CENTER>
<p><font size="5"> Big Data </span></p>
<p><font size="5"> HMEQ DATASET </font></p>
</p>
</CENTER>

<span style="border-bottom: 2px solid orange;">INTRODUCTION : </span>



In [16]:
# We import the required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

#For our classification model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

#Evaluation metrics
from sklearn.metrics import (
    confusion_matrix,           # Counts of TP, TN, FP, FN
    classification_report,      # Precision, recall, F1 summary
    roc_curve, auc,            # ROC curve and AUC
    roc_auc_score,             # AUC score directly
    ConfusionMatrixDisplay     # Visualization
)

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

In [6]:
# Load the whole dataset

df_hmeq = pd.read_csv("raw_data/hmeq.csv")
print(f"Dataset SHape : {df_hmeq.shape}")
print(f"Here are the columns : {df_hmeq.columns.to_list()}")
df_hmeq.head()

Dataset SHape : (5960, 13)
Here are the columns : ['BAD', 'LOAN', 'MORTDUE', 'VALUE', 'REASON', 'JOB', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']


,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0,39025.0,HomeImp,Other,10.5,0.0,0.0,94.366667,1.0,9.0,NaN
1,1,1300,70053.0,68400.0,HomeImp,Other,7.0,0.0,2.0,121.833333,0.0,14.0,NaN
2,1,1500,13500.0,16700.0,HomeImp,Other,4.0,0.0,0.0,149.466667,1.0,10.0,NaN
3,1,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1700,97800.0,112000.0,HomeImp,Office,3.0,0.0,0.0,93.333333,0.0,14.0,NaN


In [8]:
print("Target variable distribution:")
print(df_hmeq['BAD'].value_counts())
print(f"\nDefault rate: {df_hmeq['BAD'].mean():.1%}")
print(f"  - Non-defaults: {(df_hmeq['BAD']==0).sum():,}")
print(f"  - Defaults: {(df_hmeq['BAD']==1).sum():,}")

Target variable distribution:
BAD
0    4771
1    1189
Name: count, dtype: int64

Default rate: 19.9%
  - Non-defaults: 4,771
  - Defaults: 1,189


In [10]:
df_hmeq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5960 entries, 0 to 5959
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAD      5960 non-null   int64  
 1   LOAN     5960 non-null   int64  
 2   MORTDUE  5442 non-null   float64
 3   VALUE    5848 non-null   float64
 4   REASON   5708 non-null   object 
 5   JOB      5681 non-null   object 
 6   YOJ      5445 non-null   float64
 7   DEROG    5252 non-null   float64
 8   DELINQ   5380 non-null   float64
 9   CLAGE    5652 non-null   float64
 10  NINQ     5450 non-null   float64
 11  CLNO     5738 non-null   float64
 12  DEBTINC  4693 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 605.4+ KB


In [11]:
num_cols = df_hmeq.select_dtypes(include=['float64', 'int64']) 
cat_cols = df_hmeq.select_dtypes(include=['object']) 

print("Numeric Variables:")
print(num_cols.columns.tolist())

print("\nCategorical Variables:")
print(cat_cols.columns.tolist())

Numeric Variables:
['BAD', 'LOAN', 'MORTDUE', 'VALUE', 'YOJ', 'DEROG', 'DELINQ', 'CLAGE', 'NINQ', 'CLNO', 'DEBTINC']

Categorical Variables:
['REASON', 'JOB']


We can already see that some data is missing (row "count"). This is a serious problem because if some rows are taking into account for the traning of our model, that could alter the F1 score. Most of the data is numerical and only two columns are strings : REASON and JOB ! 

In [13]:
for col in cat_cols:
    print('We Have {} Unique Values. Values in the {} Column: {}'.format(len(df_hmeq[col].unique()),col,df_hmeq[col].unique()))
    print('__'*30)



We Have 3 Unique Values. Values in the REASON Column: ['HomeImp' nan 'DebtCon']
____________________________________________________________
We Have 7 Unique Values. Values in the JOB Column: ['Other' nan 'Office' 'Sales' 'Mgr' 'ProfExe' 'Self']
____________________________________________________________


In [15]:
print("The count of the missing values for each variable :")
df_hmeq.isnull().sum()

The count of the missing values for each variable :


BAD           0
LOAN          0
MORTDUE     518
VALUE       112
REASON      252
JOB         279
YOJ         515
DEROG       708
DELINQ      580
CLAGE       308
NINQ        510
CLNO        222
DEBTINC    1267
dtype: int64

<span style="border-bottom: 2px solid lime;">Distributions for continuous variables : </span> LOAN (with no null values), MORTDUE, VALUE, YOJ, CLAGE, DEBTINC

In [ ]:
continuous_var = ["LOAN", "MORTDUE", "VALUE", "YOJ", "CLAGE", "DEBTINC"]